In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(color_codes=True)
sns.set_context("notebook")
import tensorflow as tf
from tensorflow.contrib.rnn.python.ops import core_rnn

from matplotlib import pyplot as plt
%matplotlib inline

## Dataset
Create a dataset of sine waves. Create two groups, one with one period and another with a different period. Randomly assign different amplitudes to each wave.

In [2]:
def create_wave_sample(sample_size, num_points, period, print_stats=False):
    """
    Create a random sample of wave patterns given a sample size, number of points, and a period.
    
    Parameters:
    ----------
    sample_size: int
        Number of wave patterns to create.
    num_points: int
        How many points to use of each wave pattern.
    period: float
        The wave period to use.
    print_stats: bool
        Whether to print statistics and plot for first wave sample. Used for debugging.
        
    Returns:
    -------
    numpy.ndarray
    
    """
    wave_list = []
    count = 1
    for _ in range(sample_size):
        a = np.random.uniform(low=-10, high=10.1)
        x = np.linspace(-10*np.pi, 10*np.pi, num_points)
        wave = a*np.sin(period*x)
        wave_and_period = np.insert(float(period), 1, wave)
        wave_list.append(wave_and_period)
        if print_stats:
            print('period: {}'.format(period))
            print('sample size: {}'.format(sample_size))
            print('number of points: {}'.format(num_points))
            print('a: {}'.format(a))
            print('len(x): {}'.format(len(x)))
            print('x range: [{}, {}]'.format(min(x), max(x)))
            print('len(wave_and_period): {}'.format(len(wave_and_period)))
            print('wave range: [{}, {}]'.format(min(wave_and_period), max(wave_and_period)))
            plt.plot(x, wave)
            plt.show()
            print_stats=False
        count += 1
        
    return np.array(wave_list)

Generate two wave samples with different periods.

In [3]:
waves_1 = create_wave_sample(150, 301, 1)
waves_2 = create_wave_sample(150, 301, 2)

Concatenate both wave samples into a single sample and load into a DataFrame.

In [4]:
waves_sample = np.concatenate([waves_1, waves_2])

In [5]:
df = pd.DataFrame(waves_sample)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Columns: 302 entries, 0 to 301
dtypes: float64(302)
memory usage: 707.9 KB


Randomize the two wave samples by shuffling the DataFrame.

In [7]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)

## Model Preparation
* Split dataset into training set and test/validation set.
* Split test/validation set into test set and cross validation set.
* Create function to take random samples from the training set to train on.

Split dataset into features and labels.

In [8]:
X_array = df.ix[:, 1:].as_matrix()
y_series = df.ix[:, 0]

Cast y labels from periods to values between 0 and 1.

In [9]:
unique_ys = y_series.unique()
sorted_ys = np.sort(unique_ys)[::-1]
for i in range(len(sorted_ys)):
    y_series = y_series.replace(to_replace=sorted_ys[i], value=i)
new_unique_ys = y_series.unique()
y_array = y_series.values
print('Originals y labels: {}'.format(unique_ys))
print('New y labels: {}'.format(new_unique_ys))

Originals y labels: [ 1.  2.]
New y labels: [ 1.  0.]


Split features and labels into train and test/validation sets.

In [10]:
X_train, X_test_val, y_train, y_test_val = train_test_split(X_array, y_array, test_size=0.2, random_state=42)

Split test/validation set into test set and validation set.

In [11]:
test_val_size = int(X_test_val.shape[0] / 2)
X_test = X_test_val[0:test_val_size, :]
y_test = y_test_val[0:test_val_size]
X_val = X_test_val[test_val_size:, :]
y_val = y_test_val[test_val_size:]

Define a function to take random samples of batch size from the training data. This type of training is called bootstrap.

In [12]:
def make_batch(X, y, batch_size, back_prop_steps):
    """
    Create a random batch from the training data.
    
    Parameters:
    ----------
    X: numpy.ndarray
        Training set.
    y: numpy.ndarray
        Test set.
    batch_size: int
        Batch size.
    back_prop_stesp: int
        Back propagation size.
        
    Returns:
    -------
    numpy.ndarray, numpy.ndarray
    
    """
    num_rows, num_columns = X.shape
    bp_start_i = np.random.choice(num_columns-back_prop_steps, 1)[0]
    bp_end_i = bp_start_i + back_prop_steps
    random_indices = np.random.choice(np.arange(num_rows), batch_size, replace=False)
    X_batch = X[random_indices, bp_start_i:bp_end_i]
    y_batch = y[random_indices]
    
    return X_batch, y_batch

## TensorFlow
* Define placeholders
* Define hyperparameters
    - \# of epochs
    - learning rate
    - backprop size
    - \# of LSTM cells
* Create LSTM cell
* Create softmax layer for classification
* Define cost function
* Define training step
* Create function to check model accuracy
* Set up TF session and train model

### Define hyperparameters
Create a dictionary to store the hyperparameters

In [13]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [14]:
# Reset graph is session isn't closed
reset_graph()

In [15]:
hyperparams = {
    'num_classes': len(unique_ys),
    'batch_size': X_train.shape[0] // 10,
    'back_prop_steps': 50,
    'lstm_layers': 2,
    'lstm_cell_units': 5,
    'drop_out': 0.9,
    'clipping_ratio': 5,
    'learning_rate': 1e-4,
    'epochs': 1000
}

### Create Test and Validation Sets
Create test and validation dataset of size `batch_size`.

In [16]:
X_val, y_val = make_batch(X_val, y_val, hyperparams['batch_size'], hyperparams['back_prop_steps'])

### Define placeholders

In [17]:
X_tensor = tf.placeholder(tf.float32, [None, hyperparams['back_prop_steps']], name='Features')
y_tensor = tf.placeholder(tf.int64, [None], name='Labels')
keep_prob = tf.placeholder('float', name='Drop_Out')

### Create the LSTM cell

In [18]:
def lstm_cell(num_units):
    cell = tf.contrib.rnn.LSTMCell(num_units, state_is_tuple=True)
    return cell

In [19]:
def include_dropout(cell, prob):
    cell_culled = tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=prob)
    return cell_culled

In [20]:
def create_lstm_cell(num_units, prob, num_layers, batch_size, x):
#     cell = tf.nn.rnn_cell.LSTMCell(num_units, state_is_tuple=True)
#     cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=prob)
#    cell = tf.nn.rnn_cell.MultiRNNCell([cell]*num_layers, state_is_tuple=True)
    cell_list = [include_dropout(lstm_cell(num_units), prob) for _ in range(num_layers)]
    multi_cell = tf.contrib.rnn.MultiRNNCell(cell_list, state_is_tuple=True)
    multi_cell_culled = tf.contrib.rnn.DropoutWrapper(multi_cell, output_keep_prob=prob)
    initial_state = multi_cell_culled.zero_state(batch_size, tf.float32)
    # cell_inputs = tf.expand_dims(x, 2)
    return multi_cell_culled  # , cell_inputs, initial_state

In [21]:
# def set_lstm_state(cell, cell_inputs, initial_state, back_prop_steps):
#     cell_outputs = []
#     with tf.variable_scope('LSTM_state'):
#         for step in range(int(back_prop_steps)):
#             if step > 0:
#                 tf.get_variable_scope().reuse_variables()
#             (cell_output, state) = cell(cell_inputs[:, step, :], initial_state)
#             cell_outputs.append(cell_output)
#         output = tf.reduce_mean(tf.pack(cell_outputs), 0)
#     return output

In [22]:
def create_rnn(x, cell):
    x_list = tf.unstack(tf.expand_dims(x, axis=2), axis=1)
    outputs, _ = core_rnn.static_rnn(cell, x_list, dtype=tf.float32)
    output = outputs[-1]
    return output

### Create the softmax layer

In [23]:
def softmax_layer(num_units, num_classes, output):
    W = tf.Variable(tf.truncated_normal([num_units, num_classes], stddev=1.0),
                   name='weights', dtype=tf.float32, trainable=True)
    b = tf.Variable(tf.zeros([num_classes]),
                   name='biases', dtype=tf.float32, trainable=True)
    output_X_W = tf.matmul(output, W, name='multiply')
    logits = tf.add(output_X_W, b, name='add')
    return logits

### Define the cost function

In [24]:
def define_cost_function(logits, y, batch_size):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y, name='cross_entropy')
    cost = tf.reduce_sum(loss) / batch_size
    return cost

### Create the training step

In [25]:
def train(cost, learning_rate, clipping_ratio):
    training_variables = tf.trainable_variables()
    computed_gradients = tf.gradients(cost, training_variables)
    clipped_gradients, _ = tf.clip_by_global_norm(computed_gradients, clipping_ratio)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    # gradients = list(zip(clipped_gradients, training_variables))
    gradients = list(zip(clipped_gradients, training_variables))
    training_step = optimizer.apply_gradients(gradients)
    return training_step

### Check model accuracy during training

In [26]:
def cross_val_accuracy(logits, y):
    prediction = tf.equal(tf.argmax(logits, 1), y)
    accuracy = tf.reduce_mean(tf.cast(prediction, 'float'), name='accuracy')
    return accuracy

### Compute the final model accuracy

In [27]:
def compute_final_accuracy(X, y, batch_size, back_prop_steps):
    num_rows = X.shape[0]
    print('num_rows = {}'.format(num_rows))
    num_batches = num_rows // batch_size
    print('num_batches = {}'.format(num_batches))
    test_accuracy = []
    for i in range(int(num_batches)):
        X_batch, y_batch = make_batch(X, y, batch_size, back_prop_steps)
        result = sess.run([accuracy],
                          feed_dict = {X_tensor: X_batch,
                                       y_tensor: y_batch,
                                       keep_prob: 1})
        print('{}: accuracy = {}'.format(i, result))
        test_accuracy.append(result)
    return np.mean(test_accuracy)

## Setup the TensorFlow Graph

In [28]:
# Create LSTM cell
with tf.name_scope("LSTM_cell") as scope:
    cell = create_lstm_cell(hyperparams['lstm_cell_units'],
                            keep_prob,
                            hyperparams['lstm_layers'],
                            hyperparams['batch_size'],
                            X_tensor)
    
# # Set LSTM state
# with tf.name_scope("LSTM_state") as scope:
#     output = set_lstm_state(cell, cell_inputs, initial_state, hyperparams['back_prop_steps'])

# Create RNN
with tf.name_scope("RNN") as scope:
    output = create_rnn(X_tensor, cell)

# Create softmax layer
with tf.name_scope("softmax_layer") as scope:
    logits = softmax_layer(hyperparams['lstm_cell_units'],
                          hyperparams['num_classes'],
                          output)
    
# Define cost function
with tf.name_scope("cost_function") as scope:
    cost = define_cost_function(logits, y_tensor, hyperparams['batch_size'])
    tf.summary.scalar("cost", cost)
    
# Define training step
with tf.name_scope("training") as scope:
    training_step = train(cost, hyperparams['learning_rate'], hyperparams['clipping_ratio'])
    
# Calculate cross validation accuracy
with tf.name_scope("accuracy") as scope:
    accuracy = cross_val_accuracy(logits, y_tensor)
    tf.summary.scalar("accuracy", accuracy)

## Train the model

In [29]:
# Merge summaries for TensorBoard
merged_summaries = tf.summary.merge_all()

# Start a TensorFlow session
with tf.Session() as sess:
    
    log_directory = 'tmp/logs'
    summary_writer = tf.summary.FileWriter(log_directory, sess.graph)
    
    tf.global_variables_initializer().run()
    
    step = 0
    cost_train_ma = -np.log(1/float(hyperparams['num_classes']) + 1e-9)
    for i in range(hyperparams['epochs']):
        # Sample batch for training
        X_batch, y_batch = make_batch(X_train, y_train, 
                                      hyperparams['batch_size'], hyperparams['back_prop_steps'])
        
        # Train the model
        cost_train, _ = sess.run([cost, training_step],
                                feed_dict = {
                                    X_tensor:X_batch,
                                    y_tensor:y_batch,
                                    keep_prob:hyperparams['drop_out']
                                })
        cost_train_ma = cost_train_ma*0.99 + cost_train*0.01
        if i%100 == 0:
            
            # Perform cross validation
            result = sess.run([cost, merged_summaries, accuracy],
                             feed_dict = {
                                 X_tensor: X_val,
                                 y_tensor: y_val,
                                 keep_prob: 1
                             })
            cost_val = result[0]
            accuracy_val = result[2]
            training_step_stats = '{:d} of {:d}. Cost: [{:0.3f}, {:0.3f}], CV Cost: {:0.3f}, CV Accuracy: {:0.3f}'.format(i, hyperparams['epochs'], cost_train, cost_train_ma, cost_val, accuracy_val)
            print(training_step_stats)
            
            # Save model parameters for TensorBoard
            summary_str = result[1]
            summary_writer.add_summary(summary_str, 1)
            summary_writer.flush()
            
            step += 1
            
    final_accuracy = compute_final_accuracy(X_test, y_test, hyperparams['batch_size'], hyperparams['back_prop_steps'])
    print('The final accuracy is {:3f}.'.format(final_accuracy))
        
             

0 of 1000. Cost: [0.693, 0.693], CV Cost: 0.742, CV Accuracy: 0.417
100 of 1000. Cost: [0.732, 0.708], CV Cost: 0.725, CV Accuracy: 0.417
200 of 1000. Cost: [0.705, 0.704], CV Cost: 0.713, CV Accuracy: 0.417
300 of 1000. Cost: [0.715, 0.700], CV Cost: 0.702, CV Accuracy: 0.417
400 of 1000. Cost: [0.684, 0.692], CV Cost: 0.688, CV Accuracy: 0.500
500 of 1000. Cost: [0.675, 0.683], CV Cost: 0.670, CV Accuracy: 0.667
600 of 1000. Cost: [0.664, 0.667], CV Cost: 0.648, CV Accuracy: 0.667
700 of 1000. Cost: [0.577, 0.644], CV Cost: 0.593, CV Accuracy: 0.667
800 of 1000. Cost: [0.498, 0.597], CV Cost: 0.489, CV Accuracy: 0.917
900 of 1000. Cost: [0.389, 0.511], CV Cost: 0.378, CV Accuracy: 0.958
num_rows = 30
num_batches = 1
0: accuracy = [1.0]
The final accuracy is 1.000000.


In [ ]:
print(hyperparams['batch'])